In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [73]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} character'.format(len(text)))

Length of text: 1115394 character


In [74]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [75]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [77]:
print("{")
for char, _ in zip(char2idx, range(20)):
    print(" {:4s}: {:3d},".format(repr(char), char2idx[char]))
print(" ...\n")

{
 '\n':   0,
 ' ' :   1,
 '!' :   2,
 '$' :   3,
 '&' :   4,
 "'" :   5,
 ',' :   6,
 '-' :   7,
 '.' :   8,
 '3' :   9,
 ':' :  10,
 ';' :  11,
 '?' :  12,
 'A' :  13,
 'B' :  14,
 'C' :  15,
 'D' :  16,
 'E' :  17,
 'F' :  18,
 'G' :  19,
 ...



In [78]:
print("{} ---- characters mapped to int ---- > {}".format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [79]:
seq_length = 100
examples_per_epoch = len(text) // seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [80]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [82]:
for input_example, target_example in dataset.take(1):
    print("Input data: ", repr(''.join(idx2char[input_example.numpy()])))
    print("Target data: ", repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [83]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print(" input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print(" output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
 input: 18 ('F')
 output: 47 ('i')
Step    1
 input: 47 ('i')
 output: 56 ('r')
Step    2
 input: 56 ('r')
 output: 57 ('s')
Step    3
 input: 57 ('s')
 output: 58 ('t')
Step    4
 input: 58 ('t')
 output: 1 (' ')


In [84]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch // BATCH_SIZE

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU,
        recurrent_activation='sigmoid'
    )

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        rnn(rnn_units, 
            return_sequences=True,
            recurrent_initializer='glorot_uniform', 
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [0]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

In [89]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [90]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [92]:
sampled_indices

array([34, 25, 21, 29, 64, 24, 41, 47, 21,  9, 11, 16,  1, 56,  0,  1, 15,
       51, 57, 44, 60,  9, 24, 44, 46,  4, 41,  2, 18, 64, 46, 13,  7, 24,
        5, 43, 43, 11, 25, 48, 10, 38, 35, 58, 31, 18, 29,  1, 52, 17, 37,
       37, 12, 14,  5, 35, 61,  3, 40, 60, 31,  6, 11, 60, 47, 39, 37, 26,
       36, 25, 15, 12, 58, 43, 47, 57, 42,  6, 11, 11, 26, 29, 43,  6, 11,
       43, 53,  4, 64, 53, 59,  9, 49, 42, 49, 46, 28, 19, 17, 26])

In [93]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(''.join(idx2char[sampled_indices])))

Input: 
 " his time,\nUnto the rigour of severest law.\n\nPRINCE:\nWe still have known thee for a holy man.\nWhere'"

Next Char Predictions: 
 "VMIQzLciI3;D r\n Cmsfv3Lfh&c!FzhA-L'ee;Mj:ZWtSFQ nEYY?B'Ww$bvS,;viaYNXMC?teisd,;;NQe,;eo&zou3kdkhPGEN"


In [0]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [95]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Scalar loss: ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
Scalar loss:  4.1746297


In [0]:
model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss=loss
)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [0]:
EPOCHS=30

In [99]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/30
174/174 [==============================] - 27s 153ms/step - loss: 2.6591
Epoch 2/30
174/174 [==============================] - 24s 138ms/step - loss: 1.9359
Epoch 3/30
174/174 [==============================] - 24s 138ms/step - loss: 1.6749
Epoch 4/30
174/174 [==============================] - 24s 138ms/step - loss: 1.5319
Epoch 5/30
174/174 [==============================] - 24s 138ms/step - loss: 1.4469
Epoch 6/30
173/174 [============================>.] - ETA: 0s - loss: 1.3906

W0704 16:51:40.019556 139874215450496 util.py:244] Unresolved object in checkpoint: (root).optimizer
W0704 16:51:40.020974 139874215450496 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer
W0704 16:51:40.021946 139874215450496 util.py:244] Unresolved object in checkpoint: (root).optimizer.global_step
W0704 16:51:40.027604 139874215450496 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta1_power
W0704 16:51:40.030297 139874215450496 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta2_power
W0704 16:51:40.032487 139874215450496 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-0.embeddings
W0704 16:51:40.034448 139874215450496 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0704 16:51:40.036342 139874215450496 util.py:244] Unresolved object in checkpoint: (root).optimizer.

174/174 [==============================] - 24s 139ms/step - loss: 1.3904
Epoch 7/30
174/174 [==============================] - 24s 138ms/step - loss: 1.3445
Epoch 8/30
174/174 [==============================] - 24s 139ms/step - loss: 1.3069
Epoch 9/30
174/174 [==============================] - 24s 138ms/step - loss: 1.2745
Epoch 10/30
174/174 [==============================] - 24s 139ms/step - loss: 1.2412
Epoch 11/30
174/174 [==============================] - 24s 138ms/step - loss: 1.2090
Epoch 12/30
174/174 [==============================] - 24s 138ms/step - loss: 1.1775
Epoch 13/30
174/174 [==============================] - 24s 138ms/step - loss: 1.1450
Epoch 14/30
174/174 [==============================] - 24s 138ms/step - loss: 1.1133
Epoch 15/30
174/174 [==============================] - 24s 138ms/step - loss: 1.0811
Epoch 16/30
174/174 [==============================] - 24s 138ms/step - loss: 1.0466
Epoch 17/30
174/174 [==============================] - 24s 137ms/step - loss: 1.

In [100]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [102]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
    num_generate = 1000
    
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    
    temperature = 1.0
    
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        
        predictions = tf.squeeze(predictions, 0)
        
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1, 0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))

In [104]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: my lord.

LEONTES:
Shall I being their lordship's love,
Put up the lightning, which I have put the fairies. Pray you, go to.
You shall have a his nature, should thee mercy:
I will take thy hand doth say and stay unto Richard's love.

HENRY PERCY:
No, my good lord; it was a piteous droody throne.

GLOUCESTER:
And how my heart were flay blown paper to toward the boar that is most love,
That I shall discover all this haste gay and men
contined with peace and practise. D Englishmen:
What patiently hear my life, a goodly spirit,
Or how the joined stolen ounds restored and fort-
To know your widow's blood at the queen him first.

DUKE OF YORK:
What colours I should call?

PARIS:
Say the contrary doth make her most is true,
Even to a bough and masks their foes,
My fittest embrace and forts many mounted ruther
Will baunt with her, though I have bittenly, forseef
Thou ld 'Claughter with thy words crown'd with place,
And the occasion of a cut off thy
brgined in their clouds contented asha